In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

/tmp/ipykernel_1815/2820714271.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
# Leeftijdsgroepen: download RIVM casus-data
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2022-04-26@13-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2022-04-26@13-15.csv


loading rivm/COVID-19_casus_landelijk-2022-04-26@13-15.csv.gz


,Version,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Deceased,Week_of_death,Municipal_health_service
0,4,2022-04-26 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,NaN,GGD Amsterdam
1,4,2022-04-26 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,4,2022-04-26 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,NaN,GGD Hollands-Midden
3,4,2022-04-26 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,NaN,GGD Hollands-Noorden
4,4,2022-04-26 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,No,NaN,GGD Gelderland-Zuid


In [3]:
# Leeftijdsgroepen: download CBS bevolkingscijfers, uitgesplitst op de leeftijdscategorien zoals bij RIVM
@run
def cell():
  global bevolking
  # probeer de laatste leeftijdsgroepen op te halen bij het CBS, ververs de fallback-file 
  try:
    bevolking = CBS.bevolking(leeftijdsgroepen=True)
    bevolking.to_csv("cbs/leeftijdsgroepen_cbs.csv")
  # als het niet is gelukt de CBS file op te halen, gebruik de fallback-file en pas de index klom aan
  except Exception as e:
    print(e)
    bevolking = pd.read_csv("cbs/leeftijdsgroepen_cbs.csv")
    bevolking.set_index('Range', inplace=True)
  display(bevolking.head())

/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


,BevolkingOpDeEersteVanDeMaand,per 100k
Range,,
0-9,1757360,0.056904
10-19,1974261,0.050652
20-29,2273724,0.043981
30-39,2230329,0.044836
40-49,2127343,0.047007


In [4]:
# Leeftijdsgroepen: bereken per cohort besmettingen / opnamen / sterfte (incl. kleurcode), in aantal en per 100k. Vervang <50 en Unknown door Onbekend
@run
def cell():
  #  kolom is in version 2 per 18-1-22 toegevoegd, verwijderen:
  rivm.drop('Version', inplace=True, axis=1)
  display(rivm.head())
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  cohorten = list(bevolking.index) + ['Onbekend']

  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  factor = bevolking.to_dict()['per 100k']

  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  # bij /100k is het Totaal-veld geen optelsom maar wordt berekend obv de gehele bevolking
  totale_bevolking_per_cohort = bevolking.to_dict()['BevolkingOpDeEersteVanDeMaand']
  totale_bevolking = sum(totale_bevolking_per_cohort.values())
  keys = [ key for key in tabel["Key"]]
  totals = [ x for x in tabel["Totaal"]]
  for k in range(0, len(keys)):
    key = keys[k]
    if '100k' in key:
      abskey = key[0] + key[5:]
      kk = keys.index(abskey)
      if isinstance(totals[k], float):
        correctedtotal = totals[kk] * (100_000 / totale_bevolking)
        # print([k, totals[k], totals[kk], correctedtotal])
        totals[k] = correctedtotal
  tabel["Totaal"] = totals

  display(tabel.head())

,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Deceased,Week_of_death,Municipal_health_service
0,2022-04-26 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,NaN,GGD Amsterdam
1,2022-04-26 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2022-04-26 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,NaN,GGD Hollands-Midden
3,2022-04-26 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,NaN,GGD Hollands-Noorden
4,2022-04-26 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,No,NaN,GGD Gelderland-Zuid


,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,...,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2022-04-26,20/04-26/04,45.458608,9398.0,Positief getest,174.0,519.0,1650.0,...,288,917,972,763,1000,594,329,191,71,0
1,p001,1,2022-04-26,13/04-19/04,45.846719,22039.0,Positief getest,387.0,1227.0,3595.0,...,295,866,1000,800,996,665,331,197,66,0
2,p002,2,2022-04-26,06/04-12/04,45.418264,55826.0,Positief getest,1398.0,3206.0,8306.0,...,298,773,988,806,1000,685,319,147,49,0
3,p003,3,2022-04-26,30/03-05/04,46.348168,110784.0,Positief getest,3133.0,6686.0,15137.0,...,325,736,947,799,1000,804,425,154,46,0
4,p004,4,2022-04-26,23/03-29/03,46.758147,200084.0,Positief getest,6853.0,12863.0,25395.0,...,366,723,967,826,1000,839,551,183,46,0


In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


updating knack


...

updating: [{'Key': 1, 'Updating Leeftijdsgroep': 'false'}]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.27s/it]

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 0, update: 1, delete: 0, backoff: 0
